# EEG - Flow

## 4. ICAs



In [16]:
# from itertools import chain

from mne import read_annotations, pick_types
from mne.io import read_raw_fif, write_info, read_info
from mne.preprocessing import compute_bridged_electrodes, interpolate_bridged_electrodes, ICA
from mne.viz import set_browser_backend
from mne_icalabel import label_components
from pyprep import NoisyChannels

from eeg_flow.config import load_config
from eeg_flow.utils.annotations import merge_bad_annotations
from eeg_flow.utils.bids import get_fname, get_folder
from eeg_flow.utils.concurrency import lock_files
from eeg_flow.viz import plot_bridged_electrodes

import pandas as pd

%matplotlib qt
set_browser_backend('qt')

_, derivatives_folder, experimenter = load_config()

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [15]:
participant = 3  # int
group =  1  # int
task = "oddball"  # str
run = 1  # int

derivatives_folder_preprocessed_p = get_folder(derivatives_folder / "preprocessed", participant, group)
derivatives_folder_plots_p = get_folder(derivatives_folder / "plots", participant, group)
fname_stem = get_fname(participant, group, task, run)

#condition: do we have 2 sets of annotations?

# create locks
derivatives = (
    derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step4_first-ica.fif"),
    derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step4_second-ica.fif"),
    derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step4_icalabel.xlsx"),
    
)
locks = lock_files(*derivatives)

# load previous step
# load raw recording
raw = read_raw_fif(derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step1_raw.fif"), preload=True)
annot = read_annotations(derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step2_oddball_with_bads_annot.fif"))
info = read_info(derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step2_info.fif"))

#is this necessary here?
raw.annotations.__add__(annot)

##merge infos? if necessary?

RuntimeError: Could not lock all files. ['L:\\EEG_Flow_data\\derivatives\\preprocessed\\sub-P03-G1\\sub-P03-G1_task-oddball_run-1\\sub-P03-G1_task-oddball_run-1_step4_first-ica.fif', 'L:\\EEG_Flow_data\\derivatives\\preprocessed\\sub-P03-G1\\sub-P03-G1_task-oddball_run-1\\sub-P03-G1_task-oddball_run-1_step4_second-ica.fif'] are already in-use.

## 2.1 ICA1 For mastoids


In [4]:
raw_ica_fit1 = raw.copy()
# Filter to final BP (1, 40) Hz  ### or load the raw that was already fit at a previous stage
raw_ica_fit1.filter(
    l_freq=1.0,
    h_freq=40.0,
    picks="eeg",
    method="fir",
    phase="zero-double",
    fir_window="hamming",
    fir_design="firwin",
    pad="edge",
)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-12 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-12 dB cutoff frequency: 45.00 Hz)
- Filter length: 3381 samples (3.302 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.7s finished


<Raw | sub-P03-G1_task-oddball_run-1_step1_raw.fif, 68 x 378277 (369.4 s), ~196.3 MB, data loaded>

In [5]:
%%time 
#%%% Fit an ICA
ica1 = ICA(
    n_components=10, 
    method="picard",
    max_iter="auto",
    fit_params=dict(ortho=False, extended=True),
    random_state = 888,
)
picks = pick_types(raw_ica_fit1.info, eeg=True, exclude="bads")
ica1.fit(raw_ica_fit1, picks=picks)
# notify

2023-04-10 22:38:31,208 - numexpr.utils - INFO - Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-04-10 22:38:31,209 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 10 components
Fitting ICA took 13.3s.
CPU times: total: 49.6 s
Wall time: 13.4 s


Method,picard
Fit,54 iterations on raw data (378277 samples)
ICA components,10
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


## 2.2 ICA2 For mastoids


In [6]:
#%% Clean the other channels
# The first step is to prepare the raw object for an ICA, and for suggestions
# from ICLabel. The steps are very similar to the previous ones.
raw.drop_channels(["M1", "M2"])

<Raw | sub-P03-G1_task-oddball_run-1_step1_raw.fif, 66 x 378277 (369.4 s), ~190.5 MB, data loaded>

In [7]:
# filter
raw_ica_fit2 = raw.copy()
raw_ica_fit2.filter(
    l_freq=1.0,
    h_freq=100.0,  # Note the higher frequency
    picks=["eeg"],
    method="fir",
    phase="zero-double",
    fir_window="hamming",
    fir_design="firwin",
    pad="edge",
)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-12 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-12 dB cutoff frequency: 112.50 Hz)
- Filter length: 3381 samples (3.302 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  61 out of  61 | elapsed:    0.7s finished


<Raw | sub-P03-G1_task-oddball_run-1_step1_raw.fif, 66 x 378277 (369.4 s), ~190.5 MB, data loaded>

In [8]:
# change the reference to a common average reference (CAR)
raw_ica_fit2.set_montage(None)
raw_ica_fit2.add_reference_channels(ref_channels="CPz")
raw_ica_fit2.set_montage("standard_1020")
raw_ica_fit2.set_eeg_reference("average", projection=False)
# Note that the CAR is excluding the bad channels.


Location for this channel is unknown; consider calling set_montage() again if needed.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


<Raw | sub-P03-G1_task-oddball_run-1_step1_raw.fif, 67 x 378277 (369.4 s), ~193.5 MB, data loaded>

In [9]:
%%time 

# fit an ICA
ica2 = ICA(
    n_components=10,  # can be set to None
    method="picard",
    max_iter="auto",
    fit_params=dict(ortho=False, extended=True),
    random_state = 888,
)
picks = pick_types(raw_ica_fit2.info, eeg=True, exclude="bads")
ica2.fit(raw_ica_fit2, picks=picks)
#notify

Fitting ICA to data using 62 channels (please be patient, this may take a while)
Selecting by number: 10 components
Fitting ICA took 12.9s.
CPU times: total: 49.3 s
Wall time: 12.9 s


Method,picard
Fit,49 iterations on raw data (378277 samples)
ICA components,10
Available PCA components,62
Channel types,eeg
ICA components marked for exclusion,—


## 2.3 ICA2 icalabel


In [10]:
%%time
#%% Label components
# Let's start by getting suggestion from the ICLabel model
component_dict = label_components(raw_ica_fit2, ica2, method="iclabel")
print(component_dict)

{'y_pred_proba': array([0.9880762 , 0.99975   , 0.45581785, 0.9934921 , 0.9905507 ,
       0.9989004 , 0.9212516 , 0.9250847 , 0.4262342 , 0.90114903],
      dtype=float32), 'labels': ['eye blink', 'brain', 'muscle artifact', 'brain', 'brain', 'brain', 'eye blink', 'muscle artifact', 'other', 'muscle artifact']}
CPU times: total: 3.33 s
Wall time: 2.73 s


In [13]:
type(component_dict)

dict

In [18]:
data_icalabel = {'y_pred': component_dict['y_pred_proba'], 
                 'labels': component_dict["labels"]}
df_icalabel = pd.DataFrame.from_dict(data_icalabel)

In [29]:
#or to_csv
fname_icalabel = derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step4_icalabel.xlsx")

df_icalabel.to_excel(fname_icalabel)

In [ ]:
# save_path = os.path.join(output_prep_path, "ICALabel.txt")
# file_ica = open(save_path,"w")
# file_ica.write("i \t y_pred \t labels\n")

# for i in range(len(component_dict["y_pred_proba"])):
#     file_ica.write(str(i) + "\t" + str(component_dict["y_pred_proba"][i]) + "\t" + str(component_dict["labels"][i]) + "\n")
    
# file_ica.close() #to change file access modes

In [ ]:
# let's remove eye-blink and heart beat
labels = component_dict["labels"]
exclude = [
    k for k, name in enumerate(labels) if name in ("eye blink", "heart beat")
]

In [ ]:
# let's remove other non-brain components that occur often
_, _, _, data = _prepare_data_ica_properties(
    raw_ica_fit,
    ica,
    reject_by_annotation=True,
    reject="auto",
)

ica_data = np.swapaxes(data, 0, 1)
var = np.var(ica_data, axis=2)  # (n_components, n_epochs)
var = np.var(var.T / np.linalg.norm(var, axis=1), axis=0)
# linear fit to determine the variance thresholds
z = np.polyfit(range(0, ica.n_components_, 1), var, 1)
threshold = [z[0] * x + z[1] for x in range(0, ica.n_components_, 1)]
# add non-brain ICs below-threshold to exclude
for k, label in enumerate(labels):
    if label in ("brain", "eye blink", "heart beat"):
        continue
    if threshold[k] <= var[k]:
        continue
    exclude.append(k)
ica.exclude = exclude

## 2.4 Save derivatives

The updated annotations can now be saved alongside the selected bad channels.

In [ ]:
fname_ica1 = derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step4_ica1.fif")
fname_ica2 = derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step4_ica2.fif")

ica1.save(fname_ica1, overwrite = False)
ica2.save(fname_ica2, overwrite = False)

Regardless of the success of the task, the locks must be released.
If this step is forgotten, someone might have to remove the corresponding `.lock` file manually.

In [ ]:
for lock in locks:
    lock.release()
del locks  # delete would release anyway